### opdracht 2



In [14]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from numpy.linalg import norm
import gensim 

# preprocess de zinnen
import nltk
import re
from nltk.corpus import stopwords
from unidecode import unidecode

### make the matrix 

In [2]:
line = np.array('altijd november altijd regen altijd dit lege hart altijd'.split())
enc = OneHotEncoder()
data = enc.fit_transform(line.reshape(-1, 1))

corpus = {'altijd':1, 'dit':2, 'hart':3, 'lege':4, 'november':5, 'regen':6}
data = [corpus[k] for k in 
    'altijd november altijd regen altijd dit lege hart altijd'.split()]

### import the model

In [5]:
model_path = "BioWordVec_PubMed_MIMICIII_d200.vec.bin"
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)
#kijken of het model goed is geladen
len(model.key_to_index) #16545452

#het model positioneert de woorden in een 200-dimensionale vectorruimte
s = model['man']
type(s) #ndarray
s.shape #(200,)


(200,)

### Cosinusgelijkheid

In [11]:
def cosine_similarity(vec1, vec2):
    # # define two lists or array

    # A = np.array([2,1,2,3,2,9])
    # B = np.array([3,4,2,4,5,5])
    # bereken dot product 
    dot_product = np.dot(vec1, vec2)

    # bereken de norm van elke vector
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    # bereken cosinusgelijkheid
    similarity = dot_product / (norm_vec1 * norm_vec2)

    return similarity


vector1 = np.array([1, 2, 3])
vector2 = np.array([4, 5, 6])

resultaat = cosine_similarity(vector1, vector2)
print(f"Cosine Similarity: {resultaat}")

Cosine Similarity: 0.9746318461970762


### preprocess de zinnen

In [ ]:
# Download stopwoordenlijst (eenmalig)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
def preprocess_sentence(sentence):
    # Zet om naar kleine letters
    sentence = sentence.lower()
    
    # Verwijder de/het/een tekens
    sentence = unidecode(sentence)
    
    # Verwijder interpunctie en speciale tekens, behoud alleen letters en spaties
    sentence = re.sub(r'[^a-z\s]', '', sentence)
    
    # Verwijder stopwoorden
    stop_words = set(stopwords.words('dutch'))  # Voor Nederlands, gebruik 'english' voor Engels
    words = sentence.split()
    filtered_words = [word for word in words if word not in stop_words]
    
    # Zet de woorden terug naar een zin
    processed_sentence = ' '.join(filtered_words)
    
    return processed_sentence

# Voorbeeldgebruik
zin = "Hallo, wereld! Dit is een voorbeeldzin met diakritische tekens: naïve en résumé."
print(preprocess_sentence(zin))

hallo wereld voorbeeldzin diakritische tekens naive resume


In [19]:
def sentence_to_vector(sentence, model):
    # Preprocess de zin
    sentence = preprocess_sentence(sentence)
    
    # Split de zin in woorden
    words = sentence.split()
    
    # Haal de vectoren op voor de woorden
    word_vectors = []
    for word in words:
        if word in model:
            word_vectors.append(model[word])
    
    # Bereken het gemiddelde van de woordembeddings (indien er woorden zijn)
    if word_vectors:
        sentence_vector = np.mean(word_vectors, axis=0)
    else:
        # Als er geen bekende woorden zijn in het model, retourneer een lege vector
        sentence_vector = np.zeros(model.vector_size)
    
    return sentence_vector

In [21]:
# Voorbeeldzinnen
zin1 = "Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis."
zin2 = "Breast cancers with HER2 amplification are more aggressive, have a higher risk of CNS metastasis, and poorer prognosis."
zin3 = "Furthermore, increased CREB expression in breast tumors is associated with poor prognosis, shorter survival and higher risk of metastasis."

# Zet zinnen om naar vectoren
vec1 = sentence_to_vector(zin1, model)
vec2 = sentence_to_vector(zin2, model)
vec3 = sentence_to_vector(zin3, model)

# Bereken de cosine similarity tussen de zinnen
sim12 = cosine_similarity(vec1, vec2)
sim13 = cosine_similarity(vec1, vec3)
sim23 = cosine_similarity(vec2, vec3)

print(f"Cosine Similarity tussen zin 1 en zin 2: {sim12}")
print(f"Cosine Similarity tussen zin 1 en zin 3: {sim13}")
print(f"Cosine Similarity tussen zin 2 en zin 3: {sim23}")

Cosine Similarity tussen zin 1 en zin 2: 0.9913076162338257
Cosine Similarity tussen zin 1 en zin 3: 0.9479348063468933
Cosine Similarity tussen zin 2 en zin 3: 0.9444634318351746
